In [1]:
import os
from lxml import etree as lxml
import zipfile, itertools
import io
from officelib.xllib import *

In [2]:
os.chdir("C:\\Users\\Nathan\\Downloads")

In [3]:
def load_xml(z):
    with z.open("archive_manifest.xml", 'r') as f:
        tree = lxml.parse(f)
    return tree

def load_zip(file):
    return zipfile.ZipFile(file)

In [4]:
tree = lxml.parse("test\\ViewOnlySecurity.xml")
root = tree.getroot()
for c in root.findall("security-group"):
    if c.find("name").text == "View":
        break
else:
    raise ValueError("oops")

In [5]:
xl = Excel()
wb = None
_wb = None
for _wb in xl.Workbooks:
    if _wb.Name == "helix_security.xlsx":
        wb = _wb
        break
del _wb
if wb is None:
    wb = xl.Workbooks.Add()
    wb.Name = "helix_security.xlsx"
ws = wb.Worksheets(1)
cells = ws.Cells
cr = cells.Range

def switch_sheet(id):
    global ws, cells, cr
    ws = wb.Worksheets(id)
    cells = ws.Cells
    cr = cells.Range

In [6]:
def paste(cell, data):
    c2 = cell.GetOffset(len(data) - 1, len(data[0]) - 1)
    target = cell.Range(cell, c2)
    target.NumberFormat = "@"
    target.Value = data
    

In [31]:
out = []

def_fields_elem = root.xpath("security-group/field-security/field-type[@name=\"Artifacts\"]/field-subtype[@name=\"Default\"]/field-category[@name=\"Custom Fields\"]")[0]
for e in root.xpath("security-group/field-security/field-type[@name=\"Artifacts\"]/field-subtype[@override-default]"):
    if e.attrib['name'] == 'Default':
        continue
    thelist = []
    out.append(thelist)
    thelist.append(e.attrib['name'])
    custom_fields = e.xpath("./field-category[@name=\"Custom Fields\"]")[0]
    for field in custom_fields:
        if field.attrib['onAdd'] != field.attrib['onEdit']:
            raise ValueError("???")
        if field.attrib['onAdd'] != 'Hidden':
            thelist.append(field.text)
        
    
        

In [33]:
from itertools import zip_longest

In [34]:
data = list(zip_longest(*out))
paste(cr("A1"),data)